## Budapest ingatlanbefetetésre ajánlott kerületei

In [82]:
sale_raw<-read.csv2("idealista_buy.csv", as.is=T, sep=",")
rent_raw<-read.csv2("idealista_rent.csv", as.is=T, sep=",")

In [83]:
head(rent_raw)

elevator,rooms,address,price,size
con ascensore,NA,"Monolocale in corso di porta nuova, 52, Moscova-Repubblica, Milano",680,27
,2,"Bilocale in via privata catone, 29, Dergano, Milano",700,80
con ascensore,3,"Trilocale in via giuseppe garibaldi, 94, Cinisello, Cinisello Balsamo",800,88
con ascensore,2,"Bilocale in via luigi guanella, 4, Precotto, Milano",800,40
senza ascensore,3,"Trilocale in via privata galeno, 18, Precotto, Milano",950,85
con ascensore,2,"Bilocale in via beato angelico, 23, Città Studi, Milano",1.000,70


In [84]:
rent_raw$price<- as.numeric(gsub("\\.", "", rent_raw$price))
sale_raw$price<- as.numeric(gsub("\\.", "", sale_raw$price))

In [85]:
sale_raw$elevator[!(sale_raw$elevator  %in% c(" con ascensore"," senza ascensore"))]<-NA
rent_raw$elevator[!(rent_raw$elevator  %in% c(" con ascensore"," senza ascensore"))]<-NA

sale_raw$elevator[sale_raw$elevator == " con ascensore"]<-1
sale_raw$elevator[sale_raw$elevator == " senza ascensore"]<-0
rent_raw$elevator[rent_raw$elevator == " con ascensore"]<-1
rent_raw$elevator[rent_raw$elevator == " senza ascensore"]<-0

sale_raw$elevator<-as.numeric(sale_raw$elevator)
rent_raw$elevator<-as.numeric(rent_raw$elevator)

summary(sale_raw)

     size              elevator       address              rooms       
 Length:40744       Min.   :0.000   Length:40744       Min.   : 2.000  
 Class :character   1st Qu.:1.000   Class :character   1st Qu.: 2.000  
 Mode  :character   Median :1.000   Mode  :character   Median : 3.000  
                    Mean   :0.821                      Mean   : 3.214  
                    3rd Qu.:1.000                      3rd Qu.: 4.000  
                    Max.   :1.000                      Max.   :59.000  
                    NA's   :17318                      NA's   :2660    
     price         
 Min.   :     265  
 1st Qu.:  129000  
 Median :  199000  
 Mean   :  322237  
 3rd Qu.:  350000  
 Max.   :88888888  
                   

In [93]:
head(strsplit(sale_raw$address, ",")[3])

[[1]]
[1] "Trilocale in via privata albona" " 8"                             
[3] " Baggio"                         " Milano"

In [49]:
summary(sale_raw)

     size                       elevator       address         
 Length:40744                       :14541   Length:40744      
 Class :character                   : 2777   Class :character  
 Mode  :character    con ascensore  :19224   Mode  :character  
                     senza ascensore: 4202                     
                                                               
                                                               
                                                               
     rooms            price         
 Min.   : 2.000   Min.   :     265  
 1st Qu.: 2.000   1st Qu.:  129000  
 Median : 3.000   Median :  199000  
 Mean   : 3.214   Mean   :  322237  
 3rd Qu.: 4.000   3rd Qu.:  350000  
 Max.   :59.000   Max.   :88888888  
 NA's   :2660                       

In [53]:
sale_raw$sm<-as.numeric(as.character(sale_raw$price))/as.numeric(as.character(sale_raw$size))*1000000
rent_raw$sm<-as.numeric(as.character(rent_raw$price))/as.numeric(as.character(rent_raw$size))

Warning message:
In eval(expr, envir, enclos): NAs introduced by coercionWarning message:
In eval(expr, envir, enclos): NAs introduced by coercion

In [54]:
x <- sale_raw$sm
qnt <- quantile(x, probs=c(.25, .75), na.rm = T)
caps <- quantile(x, probs=c(.05, .95), na.rm = T)
H <- 1.5 * IQR(x, na.rm = T)
sale_raw<-sale_raw[sale_raw$sm > (qnt[1] - H),]
sale_raw<-sale_raw[sale_raw$sm < (qnt[2] + H),]

x <- rent_raw$sm
qnt <- quantile(x, probs=c(.25, .75), na.rm = T)
caps <- quantile(x, probs=c(.05, .95), na.rm = T)
H <- 1.5 * IQR(x, na.rm = T)
rent_raw<-rent_raw[rent_raw$sm > (qnt[1] - H),]
rent_raw<-rent_raw[rent_raw$sm < (qnt[2] + H),]

In [55]:
rent_aggregated<-aggregate(sm~adress3,data=rent_raw,FUN=mean)
rent_aggregated$rent_count<-aggregate(sm~adress3,data=rent_raw,FUN=length)[2]
sale_aggregated<-aggregate(sm~adress3,data=sale_raw, FUN=mean)
sale_aggregated$sale_count<-aggregate(sm~adress3,data=sale_raw,FUN=length)[2]
compare_district<-merge(rent_aggregated,sale_aggregated, by="adress3")
compare_district$sm.ratio<-compare_district$sm.y/compare_district$sm.x

In [56]:
compare_district<-compare_district[compare_district$rent_count>50 & compare_district$sale_count>50,]
compare_district[order(compare_district$sm.ratio),]

,adress3,sm.x,rent_count,sm.y,sale_count,sm.ratio
22,"8. kerület, Budapest",2961.917,581,395261.4,4765,133.4478
17,"3. kerület, Budapest",3080.003,283,459182.9,4042,149.0852
5,"13. kerület, Budapest",3185.367,839,505189.4,6729,158.5969
6,"14. kerület, Budapest",2544.96,281,418276.5,4822,164.3548
21,"7. kerület, Budapest",2995.794,923,495875.7,5371,165.5239
23,"9. kerület, Budapest",2985.728,690,495484.9,3155,165.9511
20,"6. kerület, Budapest",3464.306,1353,584643.1,5078,168.762
12,"2. kerület, Budapest",3548.513,1243,609267.5,3787,171.6966
1,"1. kerület, Budapest",3525.838,464,610442.1,1300,173.1339
4,"12. kerület, Budapest",3439.834,681,597393.6,2124,173.6693


In [57]:
rent_aggregated<-aggregate(sm~adress2,data=rent_raw,FUN=mean)
rent_aggregated$rent_count<-aggregate(sm~adress2,data=rent_raw,FUN=length)[2]
sale_aggregated<-aggregate(sm~adress2,data=sale_raw, FUN=mean)
sale_aggregated$sale_count<-aggregate(sm~adress2,data=sale_raw,FUN=length)[2]
compare_district<-merge(rent_aggregated,sale_aggregated, by="adress2")
compare_district$sm.ratio<-compare_district$sm.y/compare_district$sm.x

In [64]:
compare_district<-compare_district[compare_district$rent_count>50 & compare_district$sale_count>50,]
result<-compare_district[order(compare_district$sm.ratio),]
result

,adress2,sm.x,rent_count,sm.y,sale_count,sm.ratio
93,Józsefváros – Corvinnegyed,3510.975,177,448792.1,785,127.8255
94,Józsefváros – Csarnoknegyed,2941.567,64,381495,636,129.6911
74,Hárshegy,4492.778,53,651379.2,62,144.9836
140,Külső-Terézváros – Diplomatanegyed,3833.552,75,564397.7,356,147.2258
125,Középső-Ferencváros,3102.495,77,465824.2,577,150.145
17,Angyalföld – Lőportárdűlő,3070.982,77,462253.8,862,150.5231
116,Kissvábhegy,3861.493,63,602392.2,118,155.9998
239,Vérhalom,4066.465,82,639620.5,102,157.2915
12,Angyalföld,3096.708,151,488811,2247,157.8486
137,Külső-Erzsébetváros – Ligetváros,2826.505,65,447437.4,703,158.3006
